# 5. Composição Multi-Visualização

Ao visualizar diversos campos de dados, podemos ser tentados a usar o máximo possível de canais de codificação visual: `x`, `y`, `color`, `size`, `shape`, e assim por diante. No entanto, à medida que o número de canais de codificação aumenta, um gráfico pode rapidamente se tornar desorganizado e difícil de ler. Uma alternativa para "sobrecarregar" um único gráfico é _compor varios gráficos_ de uma maneira que facilite comparações rápidas.

Neste notebook, examinaremos uma variedade de operações para _composição de múltiplas visualizações_:

- _camada_: colocar gráficos compatíveis diretamente uns sobre os outros,
- _faceta_: particionar dados em vários gráficos, organizados em linhas ou colunas,
- _concatenar_: posicionar gráficos arbitrários dentro de um layout compartilhado, e
- _repetir_: pegar uma especificação de gráfico base e aplica-la a múltiplos campos de dados.

Veremos então como essas operações formam uma _álgebra de composição de visualização_, na qual as operações podem ser combinadas para construir uma variedade de exibições complexas de múltiplas visualizações.

_Este notebook faz parte do [currículo de visualização de dados](https://github.com/uwdata/visualization-curriculum)._

In [1]:
import pandas as pd
import altair as alt

## 5.1. Dados meteorológicos

Estaremos visualizando estatísticas meteorológicas para as cidades dos E.U.A. de Seattle e Nova York. Vamos carregar o dataset e dar uma olhada nas primeiras 10 linhas:

In [2]:
weather = 'https://cdn.jsdelivr.net/npm/vega-datasets@1/data/weather.csv'

In [3]:
df = pd.read_csv(weather)
df.head(10)

,location,date,precipitation,temp_max,temp_min,wind,weather
0,Seattle,2012-01-01,0.0,12.8,5.0,4.7,drizzle
1,Seattle,2012-01-02,10.9,10.6,2.8,4.5,rain
2,Seattle,2012-01-03,0.8,11.7,7.2,2.3,rain
3,Seattle,2012-01-04,20.3,12.2,5.6,4.7,rain
4,Seattle,2012-01-05,1.3,8.9,2.8,6.1,rain
5,Seattle,2012-01-06,2.5,4.4,2.2,2.2,rain
6,Seattle,2012-01-07,0.0,7.2,2.8,2.3,rain
7,Seattle,2012-01-08,0.0,10.0,2.8,2.0,sun
8,Seattle,2012-01-09,4.3,9.4,5.0,3.4,rain
9,Seattle,2012-01-10,1.0,6.1,0.6,3.4,rain


In [4]:
df.tail(10)

,location,date,precipitation,temp_max,temp_min,wind,weather
2912,New York,2015-12-22,4.8,15.6,11.1,3.8,fog
2913,New York,2015-12-23,29.5,17.2,8.9,4.5,fog
2914,New York,2015-12-24,0.5,20.6,13.9,4.9,fog
2915,New York,2015-12-25,2.5,17.8,11.1,0.9,fog
2916,New York,2015-12-26,0.3,15.6,9.4,4.8,drizzle
2917,New York,2015-12-27,2.0,17.2,8.9,5.5,fog
2918,New York,2015-12-28,1.3,8.9,1.7,6.3,snow
2919,New York,2015-12-29,16.8,9.4,1.1,5.3,fog
2920,New York,2015-12-30,9.4,10.6,5.0,3.0,fog
2921,New York,2015-12-31,1.5,11.1,6.1,5.5,fog


Criaremos exibições com múltiplas visualizações para examinar o clima dentro e entre as cidades.

## 5.2. Camadas

Uma das formas mais comuns de combinar múltiplos gráficos é *sobrepor* marcas umas sobre as outras. Se os domínios das escalas subjacentes forem compatíveis, podemos uni-los para formar _shared axes_ (eixos compartilhados). Se alguma das codificações `x` ou `y` não for compatível, podemos, em vez disso, criar um _dual-axis chart_ (gráfico duplo-eixo), que sobrepõe marcas usando escalas e eixos separados.

### 5.2.1 Eixos compartilhados

Comecemos plotando as temperaturas médias mínimas e máximas por mês:

In [5]:
alt.Chart(weather).mark_area().encode(
  alt.X('month(date):T'),
  alt.Y('average(temp_max):Q'),
  alt.Y2('average(temp_min):Q')
)

alt.Chart(...)

_O gráfico nos mostra os intervalos de temperatura para cada mês ao longo de todo o nosso conjunto de dados. No entanto, isso pode ser enganoso, pois agrega as medições tanto de Seattle quanto de Nova York!_

Vamos subdividir os dados por localização usando uma codificação de cor, além de ajustar a opacidade da marca para acomodar áreas sobrepostas:

In [6]:
alt.Chart(weather).mark_area(opacity=0.3).encode(
  alt.X('month(date):T'),
  alt.Y('average(temp_max):Q'),
  alt.Y2('average(temp_min):Q'),
  alt.Color('location:N')
)

alt.Chart(...)

_Podemos ver que Seattle tem um clima mais ameno: mais quente no inverno e mais fresco no verão._

Neste caso, criamos um gráfico em camadas sem recursos especiais, apenas subdividindo as áreas por cor. Embora o gráfico acima nos mostre os intervalos de temperatura, também podemos querer enfatizar o meio do intervalo.

Vamos criar um gráfico de linhas mostrando a média dos pontos médios de temperatura. Usaremos uma transformação `calculate` para calcular os pontos médios entre as temperaturas mínimas e máximas diárias:

In [7]:
alt.Chart(weather).mark_line().transform_calculate(
  temp_mid='(+datum.temp_min + +datum.temp_max) / 2'
).encode(
  alt.X('month(date):T'),
  alt.Y('average(temp_mid):Q'),
  alt.Color('location:N')
)

alt.Chart(...)

_Observação_: note o uso de `+datum.temp_min` dentro da transformação `calculate`. Como estamos carregando os dados diretamente de um arquivo CSV sem nenhuma instrução especial de análise, os valores de temperatura podem estar internamente representados como strings. Adicionar `+` antes do valor força sua interpretação como um número.  

Nós gostaríamos agora de combinar esses gráficos sobrepondo as linhas dos pontos médios às áreas das faixas. Usando a sintaxe `chart1 + chart2`, podemos especificar que queremos um novo gráfico em camadas, no qual `chart1` é a primeira camada e `chart2` é uma segunda camada desenhada por cima:

In [8]:
tempMinMax = alt.Chart(weather).mark_area(opacity=0.3).encode(
  alt.X('month(date):T'),
  alt.Y('average(temp_max):Q'),
  alt.Y2('average(temp_min):Q'),
  alt.Color('location:N')
)

tempMid = alt.Chart(weather).mark_line().transform_calculate(
  temp_mid='(+datum.temp_min + +datum.temp_max) / 2'
).encode(
  alt.X('month(date):T'),
  alt.Y('average(temp_mid):Q'),
  alt.Color('location:N')
)

tempMinMax + tempMid

alt.LayerChart(...)

_Agora temos um gráfico com múltiplas camadas! No entanto, o título do eixo y (embora informativo) ficou um pouco longo e desajeitado..._

Vamos personalizar nossos eixos para limpar a apresentação do gráfico. Se definirmos um título personalizado em uma das camadas para o eixo, ele será automaticamente usado como um título de eixo compartilhado para todas as camadas:

In [9]:
tempMinMax = alt.Chart(weather).mark_area(opacity=0.3).encode(
  alt.X('month(date):T', title=None, axis=alt.Axis(format='%b')),
  alt.Y('average(temp_max):Q', title='Avg. Temperature °C'),
  alt.Y2('average(temp_min):Q'),
  alt.Color('location:N')
)

tempMid = alt.Chart(weather).mark_line().transform_calculate(
  temp_mid='(+datum.temp_min + +datum.temp_max) / 2'
).encode(
  alt.X('month(date):T'),
  alt.Y('average(temp_mid):Q'),
  alt.Color('location:N')
)

tempMinMax + tempMid

alt.LayerChart(...)

_O que acontece se ambas as camadas tiverem títulos personalizados para o eixo? Modifique o código acima para descobrir..._

Acima, usamos o operador `+`, um atalho conveniente para o método `layer` do Altair. Podemos gerar um gráfico em camadas idêntico usando o método `layer` diretamente:

In [10]:
alt.layer(tempMinMax, tempMid)

alt.LayerChart(...)

Observe que a ordem dos elementos em uma camada importa, pois as camadas subsequentes serão desenhadas sobre as anteriores. _Tente trocar a ordem dos gráficos nas células acima. O que acontece? (Dica: observe atentamente a cor das marcas `line`.)_

### 5.2.2 Gráficos duplo-eixo

_Seattle tem a reputação de ser uma cidade chuvosa. Isso é merecido?_

Vamos observar a preciptação junto a temperatura para aprendermos mais. Primeiro vamos criar uma vizualização base que mostra a precipitação média mensal em Seatle:

In [ ]:
alt.Chart(weather).transform_filter(
  'datum.location == "Seattle"'
).mark_line(
  interpolate='monotone',
  stroke='grey'
).encode(
  alt.X('month(date):T', title=None),
  alt.Y('average(precipitation):Q', title='Precipitation')
)

alt.Chart(...)

Para facilitar a comparação com os dados de temperatura, criaremos um novo gráfico em camadas. Abaixo vemos o que acontece se tentarmos colocar os gráficos em camadas como fizemos anteriormente:

In [ ]:
tempMinMax = alt.Chart(weather).transform_filter(
  'datum.location == "Seattle"'
).mark_area(opacity=0.3).encode(
  alt.X('month(date):T', title=None, axis=alt.Axis(format='%b')),
  alt.Y('average(temp_max):Q', title='Avg. Temperature °C'),
  alt.Y2('average(temp_min):Q')
)

precip = alt.Chart(weather).transform_filter(
  'datum.location == "Seattle"'
).mark_line(
  interpolate='monotone',
  stroke='grey'
).encode(
  alt.X('month(date):T'),
  alt.Y('average(precipitation):Q', title='Precipitation')
)

alt.layer(tempMinMax, precip)

alt.LayerChart(...)

_Os valores de precipitação utilizam um intervalo muito menor no eixo y do que as temperaturas!_

Por padrão, gráficos em camadas usam um *domínio compartilhado*: os valores para o eixo x ou y são combinados entre todas as camadas para determinar a extensão compartilhada. Esse comportamento padrão assume que os valores das camadas têm as mesmas unidades. No entanto, isso não se aplica a este exemplo, pois estamos combinando valores de temperatura (graus Celsius) com valores de precipitação (polegadas)!

Se quisermos usar escalas diferentes no eixo y, precisamos especificar como queremos que o Altair *resolva* os dados entre as camadas. Neste caso, queremos resolver os domínios de escala (`scale`) do eixo y para serem independentes (`independent`), em vez de usar um domínio compartilhado (`shared`). O objeto gráfico (`Chart`) produzido por um operador de camada inclui um método `resolve_scale`, com o qual podemos especificar a resolução desejada:


In [ ]:
tempMinMax = alt.Chart(weather).transform_filter(
  'datum.location == "Seattle"'
).mark_area(opacity=0.3).encode(
  alt.X('month(date):T', title=None, axis=alt.Axis(format='%b')),
  alt.Y('average(temp_max):Q', title='Avg. Temperature °C'),
  alt.Y2('average(temp_min):Q')
)

precip = alt.Chart(weather).transform_filter(
  'datum.location == "Seattle"'
).mark_line(
  interpolate='monotone',
  stroke='grey'
).encode(
  alt.X('month(date):T'),
  alt.Y('average(precipitation):Q', title='Precipitation')
)

alt.layer(tempMinMax, precip).resolve_scale(y='independent')

alt.LayerChart(...)

_Agora podemos ver que o outono é a estação mais chuvosa em Seattle (com pico em novembro), complementada pelos verões secos._

Você pode ter notado alguma redundância nas nossas especificações de gráfico acima: ambas usam o mesmo conjunto de dados e o mesmo filtro para exibir apenas Seattle. Se desejar, você pode simplificar um pouco o código fornecendo os dados e a transformação de filtro para o gráfico em camadas de nível superior. As camadas individuais irão então herdar os dados se não tiverem suas próprias definições de dados:

In [ ]:
tempMinMax = alt.Chart().mark_area(opacity=0.3).encode(
  alt.X('month(date):T', title=None, axis=alt.Axis(format='%b')),
  alt.Y('average(temp_max):Q', title='Avg. Temperature °C'),
  alt.Y2('average(temp_min):Q')
)

precip = alt.Chart().mark_line(
  interpolate='monotone',
  stroke='grey'
).encode(
  alt.X('month(date):T'),
  alt.Y('average(precipitation):Q', title='Precipitation')
)

alt.layer(tempMinMax, precip, data=weather).transform_filter(
  'datum.location == "Seattle"'
).resolve_scale(y='independent')

alt.LayerChart(...)

Embora gráficos com eixos duplos possam ser úteis, _eles são frequentemente propensos à interpretações errôneas_, pois as diferentes unidades e escalas dos eixos podem ser incomensuráveis. Quando possível, você pode considerar transformações que mapeiem diferentes campos de dados para unidades compartilhadas, como, por exemplo, mostrando [quantis](https://pt.wikipedia.org/wiki/Quantil) ou mudança percentual relativa.

## 5.3. Facetamento

*Facetar* envolve subdividir um conjunto de dados em grupos e criar um gráfico separado para cada grupo. Em notebooks anteriores, aprendemos a criar gráficos facetados usando os canais de codificação `row` e `column`. Primeiro, revisaremos esses canais e, em seguida, mostraremos como eles são casos específicos do operador mais geral `facet`.

Vamos começar com um histograma básico dos valores de temperatura máxima em Seattle (cidade estadunidense):

In [15]:
alt.Chart(weather).mark_bar().transform_filter(
  'datum.location == "Seattle"'
).encode(
  alt.X('temp_max:Q', bin=True, title='Temperature (°C)'),
  alt.Y('count():Q')
)

alt.Chart(...)

_Como esse perfil de temperatura muda dependendo do tempo de um determinado dia – ou seja, se houve garoa, neblina, chuva, neve ou sol?_  

Vamos usar o canal de codificação `column` para facetar os dados por tipo de clima. Também podemos usar `color` como uma codificação redundante, aplicando uma escala de cores personalizada:

In [16]:
colors = alt.Scale(
  domain=['drizzle', 'fog', 'rain', 'snow', 'sun'],
  range=['#aec7e8', '#c7c7c7', '#1f77b4', '#9467bd', '#e7ba52']
)

alt.Chart(weather).mark_bar().transform_filter(
  'datum.location == "Seattle"'
).encode(
  alt.X('temp_max:Q', bin=True, title='Temperature (°C)'),
  alt.Y('count():Q'),
  alt.Color('weather:N', scale=colors),
  alt.Column('weather:N')
).properties(
  width=150,
  height=150
)

alt.Chart(...)

_Não surpreendentemente, os raros dias de neve concentram-se nas temperaturas mais frias, seguidos pelos dias chuvosos e com neblina. Dias ensolarados são mais quentes e, apesar dos estereótipos sobre Seattle, são os mais numerosos. Mas, como qualquer morador de Seattle pode confirmar, a garoa aparece de vez em quando, independentemente da temperatura!_  

Além dos canais de codificação `row` e `column` *dentro* de uma definição de gráfico, podemos pegar uma definição básica de gráfico e aplicar o facetamento usando um operador explícito `facet`.

Vamos recriar o gráfico acima, mas desta vez usando `facet`. Começamos com a mesma definição básica do histograma, mas removemos a fonte de dados, a transformação de filtro e o canal `column`. Em seguida, invocamos o método `facet`, passando os dados e especificando que o facetamento deve ocorrer em colunas de acordo com o campo `weather`. O método `facet` aceita tanto argumentos `row` quanto `column`. Ambos podem ser usados juntos para criar uma grade 2D de gráficos facetados.

Por fim, incluímos nossa transformação de filtro, aplicando-a ao gráfico facetado de nível superior. Embora pudéssemos aplicar a transformação de filtro à definição do histograma como antes, isso é ligeiramente menos eficiente. Em vez de filtrar os valores "New York" dentro de cada célula do facetamento, aplicar o filtro ao gráfico facetado informa ao Vega-Lite que podemos remover aqueles valores antecipadamente, antes da subdivisão por facetas.

In [17]:
colors = alt.Scale(
  domain=['drizzle', 'fog', 'rain', 'snow', 'sun'],
  range=['#aec7e8', '#c7c7c7', '#1f77b4', '#9467bd', '#e7ba52']
)

alt.Chart().mark_bar().encode(
  alt.X('temp_max:Q', bin=True, title='Temperature (°C)'),
  alt.Y('count():Q'),
  alt.Color('weather:N', scale=colors)
).properties(
  width=150,
  height=150
).facet(
  data=weather,
  column='weather:N'
).transform_filter(
  'datum.location == "Seattle"'
)

alt.FacetChart(...)

Dado todo o código extra acima, por que desejaríamos usar um operador `facet` explicitamente? Para gráficos básicos, certamente devemos usar os canais de codificação `column` ou `row` se pudermos. No entanto, usar o operador `facet` explicitamente é útil quando queremos facetar visualizações compostas, como gráficos sobrepostos.

Vamos revisitar nossos gráficos de temperatura sobrepostos de antes. Em vez de traçar os dados de Nova York e Seattle no mesmo gráfico, vamos separá-los em facetas distintas. As definições individuais dos gráficos são quase as mesmas de antes: um gráfico de área e um gráfico de linha. A única diferença é que, desta vez, não passaremos os dados diretamente aos construtores dos gráficos; aguardaremos e os passaremos ao operador `facet` mais adiante. Podemos sobrepor os gráficos assim como antes e, em seguida, invocar `facet` no objeto do gráfico sobreposto, passando os dados e especificando facetas `column` com base no campo `location`:

In [18]:
tempMinMax = alt.Chart().mark_area(opacity=0.3).encode(
  alt.X('month(date):T', title=None, axis=alt.Axis(format='%b')),
  alt.Y('average(temp_max):Q', title='Avg. Temperature (°C)'),
  alt.Y2('average(temp_min):Q'),
  alt.Color('location:N')
)

tempMid = alt.Chart().mark_line().transform_calculate(
  temp_mid='(+datum.temp_min + +datum.temp_max) / 2'
).encode(
  alt.X('month(date):T'),
  alt.Y('average(temp_mid):Q'),
  alt.Color('location:N')
)

alt.layer(tempMinMax, tempMid).facet(
  data=weather,
  column='location:N'
)

alt.FacetChart(...)

Os gráficos facetados que vimos até agora utilizam os mesmos domínios de escala nos eixos ao longo das células facetadas. Esse padrão de usar escalas e eixos *compartilhados* ajuda a comparar os valores com precisão. No entanto, em alguns casos, pode ser desejável dimensionar cada gráfico de forma independente, por exemplo, se o intervalo de valores em cada célula for significativamente diferente.

Semelhante aos gráficos sobrepostos, gráficos facetados também suportam _resolving_ (resolver) a escalas ou eixos independentes entre os gráficos. Vamos ver o que acontece se chamarmos o método `resolve_axis` para solicitar `independent` y-axes (eixos y independentes):

In [19]:
tempMinMax = alt.Chart().mark_area(opacity=0.3).encode(
  alt.X('month(date):T', title=None, axis=alt.Axis(format='%b')),
  alt.Y('average(temp_max):Q', title='Avg. Temperature (°C)'),
  alt.Y2('average(temp_min):Q'),
  alt.Color('location:N')
)

tempMid = alt.Chart().mark_line().transform_calculate(
  temp_mid='(+datum.temp_min + +datum.temp_max) / 2'
).encode(
  alt.X('month(date):T'),
  alt.Y('average(temp_mid):Q'),
  alt.Color('location:N')
)

alt.layer(tempMinMax, tempMid).facet(
  data=weather,
  column='location:N'
).resolve_axis(y='independent')

alt.FacetChart(...)

_O gráfico acima parece quase inalterado, mas o _plot_ de Seattle agora inclui seu próprio eixo._

E se, em vez disso, chamarmos `resolve_scale` para resolver os domínios das escalas subjacentes?

In [20]:
tempMinMax = alt.Chart().mark_area(opacity=0.3).encode(
  alt.X('month(date):T', title=None, axis=alt.Axis(format='%b')),
  alt.Y('average(temp_max):Q', title='Avg. Temperature (°C)'),
  alt.Y2('average(temp_min):Q'),
  alt.Color('location:N')
)

tempMid = alt.Chart().mark_line().transform_calculate(
  temp_mid='(+datum.temp_min + +datum.temp_max) / 2'
).encode(
  alt.X('month(date):T'),
  alt.Y('average(temp_mid):Q'),
  alt.Color('location:N')
)

alt.layer(tempMinMax, tempMid).facet(
  data=weather,
  column='location:N'
).resolve_scale(y='independent')

alt.FacetChart(...)

_Agora vemos células facetadas com diferentes domínios de escalas de eixo. Neste caso, usar escalas independentes parece uma má ideia! Os domínios não são tão diferentes, e alguém poderia se enganar achando que Nova York e Seattle têm temperaturas máximas de verão semelhantes._

Para pegar um clichê: só porque você *pode* fazer algo, não significa que você *deve*...

## 5.4 Concatenação

O facetamento cria [pequenos múltiplos](https://en.wikipedia.org/wiki/Small_multiple) gráficos que mostram subdivisões separadas dos dados. No entanto, podemos desejar criar uma exibição com múltiplas visualizações que mostrem diferentes perspectivas do mesmo conjunto de dados (não subconjuntos) ou visualizações que envolvam conjuntos de dados diferentes.

O Altair oferece operadores de concatenação para combinar gráficos arbitrários em um gráfico composto. O operador `hconcat`(abreviado como `|`) realiza a concatenação horizontal, enquanto o operador `vconcat`(abreviado como `&`) realiza a concatenação vertical.

Vamos começar com um gráfico de linha básico que mostra a temperatura máxima média por mês para Nova York e Seattle, semelhante ao que vimos antes:

In [ ]:
alt.Chart(weather).mark_line().encode(
  alt.X('month(date):T', title=None),
  alt.Y('average(temp_max):Q'),
  color='location:N'
)

alt.Chart(...)

*E se quisermos comparar não apenas a temperatura ao longo do tempo, mas também os níveis de precipitação e vento?*

Vamos criar um gráfico concatenado composto por três visualizações. Começaremos definindo uma "base" de gráfico que contém todos os aspectos que devem ser compartilhados pelas três visualizações. Em seguida, podemos modificar essa base para criar variantes personalizadas, com diferentes codificações do eixo y para os campos `temp_max`, `precipitation` e `wind`. Depois, podemos concatená-los usando o operador de pipe(`|`), que é uma abreviação para concatenação horizontal:

In [ ]:
base = alt.Chart(weather).mark_line().encode(
  alt.X('month(date):T', title=None),
  color='location:N'
).properties(
  width=240,
  height=180
)

temp = base.encode(alt.Y('average(temp_max):Q'))
precip = base.encode(alt.Y('average(precipitation):Q'))
wind = base.encode(alt.Y('average(wind):Q'))

temp | precip | wind

alt.HConcatChart(...)

Alternativamente, poderíamos usar o método mais explícito `alt.hconcat()` em vez do operador de pipe (`|`). Tente reescrever o código acima para usar `hconcat`.

A concatenação vertical funciona de forma semelhante à concatenação horizontal. Usando o operador `&` (ou o método `alt.vconcat`), modifique o código para usar uma ordenação vertical em vez de horizontal.

Por fim, observe que a concatenação horizontal e vertical pode ser combinada. O que acontece se você escrever algo como `(temp | precip) & wind`?

*Observação*: Note a importância dos parênteses... o que acontece se você removê-los? Lembre-se de que esses operadores sobrecarregados ainda estão sujeitos às [regras de precedência do Python](https://docs.python.org/3/reference/expressions.html#operator-precedence), e, portanto, a concatenação vertical com `&` terá precedência sobre a concatenação horizontal com `|`!

Como veremos mais adiante, os operadores de concatenação permitem combinar quaisquer gráficos em um painel de visualização multiview!

## 5.5 Repetir

Os operadores de concatenação acima são bastante gerais, permitindo que gráficos arbitrários sejam compostos. No entanto, o exemplo acima ainda era um pouco prolixo: temos três gráficos muito semelhantes, mas precisamos defini-los separadamente e, em seguida, concatená-los.

Para casos onde apenas uma ou duas variáveis estão mudando, o operador `repeat` oferece um atalho conveniente para criar múltiplos gráficos. Dada uma especificação de *modelo* com algumas variáveis livres, o operador repeat cria um gráfico para cada atribuição especificada a essas variáveis.

Vamos recriar nosso exemplo de concatenação acima usando o operador `repeat`. O único aspecto que muda entre os gráficos é a escolha do campo de dados para o canal de codificação `y`. Para criar uma especificação de modelo, podemos usar a *variável* `alt.repeat('column')` como nosso campo para o eixo y. Este código simplesmente declara que queremos usar a variável atribuída ao repetidor de `column`, que organiza os gráficos repetidos em uma direção horizontal. (Como o repetidor fornece apenas o nome do campo, precisamos especificar o tipo de dado do campo separadamente como `type='quantitative'`.)

Em seguida, invocamos o método `repeat`, passando os nomes dos campos de dados para cada coluna:

In [23]:
alt.Chart(weather).mark_line().encode(
  alt.X('month(date):T',title=None),
  alt.Y(alt.repeat('column'), aggregate='average', type='quantitative'),
  color='location:N'
).properties(
  width=240,
  height=180
).repeat(
  column=['temp_max', 'precipitation', 'wind']
)

alt.RepeatChart(...)

A repetição é suportada tanto para colunas quanto para linhas. _O que acontece se você modificar o código acima para usar `row` (linha) em vez de `column` (coluna)?_

Também podemos usar a repetição de `row` e `column` juntas! Uma visualização comum para análise exploratória de dados é a [scatter plot matrix (or SPLOM)](https://en.wikipedia.org/wiki/Scatter_plot#Scatterplot_matrices). Dada uma coleção de variáveis a serem analisadas, uma SPLOM fornece uma grade com todos os gráficos de dispersão possíveis entre essas variáveis, permitindo avaliar associações potenciais.

Vamos usar o operador `repeat` para criar uma SPLOM para os campos `temp_max`, `precipitation` e `wind`. Primeiro, criamos nossa especificação de modelo, com variáveis de repetição para os campos de dados do eixo x e y. Em seguida, invocamos o `repeat`, passando arrays de nomes de campos a serem usados tanto para `row` quanto para `column`. O Altair então gerará o [produto cruzado (ou produto cartesiano)](https://en.wikipedia.org/wiki/Cartesian_product) para criar o espaço completo de gráficos repetidos.

In [24]:
alt.Chart().mark_point(filled=True, size=15, opacity=0.5).encode(
  alt.X(alt.repeat('column'), type='quantitative'),
  alt.Y(alt.repeat('row'), type='quantitative')
).properties(
  width=150,
  height=150
).repeat(
  data=weather,
  row=['temp_max', 'precipitation', 'wind'],
  column=['wind', 'precipitation', 'temp_max']
).transform_filter(
  'datum.location == "Seattle"'
)

alt.RepeatChart(...)

_Observando esses gráficos, não parece haver uma associação forte entre precipitação e vento, embora possamos ver que eventos extremos de vento e precipitação ocorrem em faixas de temperatura semelhantes (~5-15° C). No entanto, essa observação não é particularmente surpreendente: se revisitarmos nosso histograma no início da seção de facetas, podemos ver claramente que os dias com temperaturas máximas na faixa de 5-15° C são os mais comuns._

*Modifique o código acima para obter uma melhor compreensão da repetição de gráficos. Tente adicionar outra variável (`temp_min`) à SPLOM. O que acontece se você reorganizar a ordem dos nomes de campos nos parâmetros `row` ou `column` para o operador `repeat`?*

_Por fim, para realmente apreciar o que o operador `repeat` oferece, reserve um momento para imaginar como você recriaria a SPLOM acima usando apenas `hconcat` e `vconcat`._

## 5.6 Uma Álgebra de Composição de Visualizações

Juntos, os operadores de composição `layer`, `facet`, `concat`, e `repeat` formam uma *álgebra de composição de visualizações*: esses diferentes operadores podem ser combinados para criar uma variedade de visualizações multi-visão.

Como exemplo, vamos começar com dois gráficos básicos: um histograma e uma linha simples (uma única marca `rule`) que representa a média global.

In [25]:
basic1 = alt.Chart(weather).transform_filter(
  'datum.location == "Seattle"'
).mark_bar().encode(
  alt.X('month(date):O'),
  alt.Y('average(temp_max):Q')
)

basic2 = alt.Chart(weather).transform_filter(
  'datum.location == "Seattle"'
).mark_rule(stroke='firebrick').encode(
  alt.Y('average(temp_max):Q')
)

basic1 | basic2

alt.HConcatChart(...)

Podemos então combinar os dois gráficos usando o operador `layer` e, em seguida, usar `repeat` para replicar esse gráfico em camadas para exibir histogramas com médias sobrepostas para múltiplos campos:

In [26]:
alt.layer(
  alt.Chart().mark_bar().encode(
    alt.X('month(date):O', title='Month'),
    alt.Y(alt.repeat('column'), aggregate='average', type='quantitative')
  ),
  alt.Chart().mark_rule(stroke='firebrick').encode(
    alt.Y(alt.repeat('column'), aggregate='average', type='quantitative')
  )
).properties(
  width=200,
  height=150
).repeat(
  data=weather,
  column=['temp_max', 'precipitation', 'wind']
).transform_filter(
  'datum.location == "Seattle"'
)

alt.RepeatChart(...)

Focando apenas nos operadores de composição de múltiplas visualizações, o modelo para a visualização acima é:

```
repeat(column=[...])
|- layer
   |- basic1
   |- basic2
```

Agora, vamos explorar como podemos aplicar *todos* os operadores dentro de um [dashboard](https://en.wikipedia.org/wiki/Dashboard_%28business%29) final que oferece uma visão geral do clima de Seattle. Combinaremos o SPLOM e os histogramas facetados das seções anteriores com os histogramas repetidos acima.

In [27]:
splom = alt.Chart().mark_point(filled=True, size=15, opacity=0.5).encode(
  alt.X(alt.repeat('column'), type='quantitative'),
  alt.Y(alt.repeat('row'), type='quantitative')
).properties(
  width=125,
  height=125
).repeat(
  row=['temp_max', 'precipitation', 'wind'],
  column=['wind', 'precipitation', 'temp_max']
)

dateHist = alt.layer(
  alt.Chart().mark_bar().encode(
    alt.X('month(date):O', title='Month'),
    alt.Y(alt.repeat('row'), aggregate='average', type='quantitative')
  ),
  alt.Chart().mark_rule(stroke='firebrick').encode(
    alt.Y(alt.repeat('row'), aggregate='average', type='quantitative')
  )
).properties(
  width=175,
  height=125
).repeat(
  row=['temp_max', 'precipitation', 'wind']
)

tempHist = alt.Chart(weather).mark_bar().encode(
  alt.X('temp_max:Q', bin=True, title='Temperature (°C)'),
  alt.Y('count():Q'),
  alt.Color('weather:N', scale=alt.Scale(
    domain=['drizzle', 'fog', 'rain', 'snow', 'sun'],
    range=['#aec7e8', '#c7c7c7', '#1f77b4', '#9467bd', '#e7ba52']
  ))
).properties(
  width=115,
  height=100
).facet(
  column='weather:N'
)

alt.vconcat(
  alt.hconcat(splom, dateHist),
  tempHist,
  data=weather,
  title='Seattle Weather Dashboard'
).transform_filter(
  'datum.location == "Seattle"'
).resolve_legend(
  color='independent'
).configure_axis(
  labelAngle=0
)

alt.VConcatChart(...)

O modelo de composição completo para este dashboard é:

```
vconcat
|- hconcat
|  |- repeat(row=[...], column=[...])
|  |  |- splom base chart
|  |- repeat(row=[...])
|     |- layer
|        |- dateHist base chart 1
|        |- dateHist base chart 2
|- facet(column='weather')
   |- tempHist base chart
```

_Ufa!_ O dashboard também inclui algumas personalizações para melhorar o layout:

- Ajustamos as propriedades `width` e `height` dos gráficos para auxiliar no alinhamento e garantir que a visualização completa caiba na tela.
- Adicionamos `resolve_legend(color='independent')` para garantir que a legenda de cores esteja diretamente associada aos histogramas coloridos por temperatura. Caso contrário, a legenda será aplicada ao dashboard como um todo.
- Usamos `configure_axis(labelAngle=0)` para garantir que as legendas dos eixos não sejam rotacionadas. Isso ajuda a manter o alinhamento correto entre os gráficos de dispersão na SPLOM e os histogramas por mês à direita.

_Tente remover ou modificar qualquer uma dessas ajustes e veja como o layout do dashboard responde!_

Este dashboard pode ser reutilizado para mostrar dados de outras localizações ou de outros conjuntos de dados. _Atualize o dashboard para mostrar os padrões climáticos de Nova York, em vez de Seattle._

## 5.7 Sumário

Para mais detalhes sobre composição de múltiplas visualizações, incluindo controle sobre o espaçamento entre subgráficos e rótulos de cabeçalho, consulte a [Altair Compound Charts documentation](https://altair-viz.github.io/user_guide/compound_charts.html).

Agora que vimos como compor múltiplas visualizações, estamos prontos para colocá-las em prática. Além de apresentar dados de forma estática, múltiplas visualizações podem permitir uma exploração interativa multidimensional. Por exemplo, usando _seleções vinculadas_, podemos destacar pontos em uma visualização para ver os valores correspondentes sendo realçados em outras visualizações.

No próximo notebook, exploraremos como criar *seleções interativas* tanto para gráficos individuais quanto para composições de múltiplas visualizações.